<a href="https://colab.research.google.com/github/Anilesh05/Big_Data_Laboratory/blob/main/Matrix_Multiplication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing jdk using apt package manager
## Downloading and installing hadoop  

In [1]:
!apt-get install openjdk-8-jdk
!wget https://dlcdn.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
!tar fx hadoop-3.3.6.tar.gz
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["HADOOP_HOME"] = "/content/hadoop-3.3.6"
!ln -s /content/hadoop-3.3.6/bin/* /usr/bin

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java libatk-wrapper-java-jni libfontenc1
  libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libice-dev librsvg2-common
  libsm-dev libxkbfile1 libxt-dev libxtst6 libxxf86dga1 openjdk-8-jdk-headless openjdk-8-jre
  openjdk-8-jre-headless x11-utils
Suggested packages:
  gvfs libice-doc libsm-doc libxt-doc openjdk-8-demo openjdk-8-source visualvm libnss-mdns
  fonts-nanum fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei
  fonts-indic mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java libatk-wrapper-java-jni libfontenc1
  libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libice-dev librsvg2-common
  libsm-dev libxkbfile1 libxt-dev libxtst6 libxxf86dga1 openjdk-

In [2]:
!apt-get install dos2unix

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  dos2unix
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 384 kB of archives.
After this operation, 1,367 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 dos2unix amd64 7.4.2-2 [384 kB]
Fetched 384 kB in 0s (913 kB/s)
Selecting previously unselected package dos2unix.
(Reading database ... 122686 files and directories currently installed.)
Preparing to unpack .../dos2unix_7.4.2-2_amd64.deb ...
Unpacking dos2unix (7.4.2-2) ...
Setting up dos2unix (7.4.2-2) ...
Processing triggers for man-db (2.10.2-1) ...


In [3]:
!hdfs dfs -mkdir input

# ***Create files in desktop***



In [ ]:
(mapper.py)

#!/usr/bin/env python
# map function for matrix multiply
#Input file assumed to have lines of the form "A,i,j,x", where i is the row index, j is the column index, and x is the value in row i, column j of A. Entries of A are followed by lines of the form "B,i,j,x" for the matrix B.
#It is assumed that the matrix dimensions are such that the product A*B exists.

#Input arguments:
#m should be set to the number of rows in A, p should be set to the number of columns in B.

import sys
import string
import numpy

#number of rows in A
m = 2

#number of columns in B
p = 3


# input comes from STDIN (stream data that goes to the program)
for line in sys.stdin:

  #Remove leading and trailing whitespace
  line = line.strip()

  #Split line into array of entry data
  entry = line.split(",")

  # Set row, column, and value for this entry
  row = int(entry[1])
  col = int(entry[2])
  value = float(entry[3])

  #If this is an entry in matrix A...
  if (entry[0] == "A"):

    #Generate the necessary key-value pairs
    for k in range(p):
      print('{0:d},{1:d}\tA,{2:d},{3:f}'.format(row, k, col, value))

  #Otherwise, if this is an entry in matrix B...
  else:

    #Generate the necessary key-value pairs
    for k in range(m):
      print('{0:d},{1:d}\tB,{2:d},{3:f}'.format(k, col, row, value))

In [ ]:
(reducer.py)


#!/usr/bin/env python
#Reduce function for computing matrix multiply A*B

#Input arguments:
#variable n should be set to the inner dimension of the matrix product (i.e., the number of columns of A/rows of B)

import sys
import string
import numpy

#number of columns of A/rows of B
n = 5

#Create data structures to hold the current row/column values
current_key = None
current_res = 0.0
value_dict = dict()

# input comes from STDIN (stream data that goes to the program)
for line in sys.stdin:

  #Remove leading and trailing whitespace
  line = line.strip()

  #Get key/value
  key, value = line.split('\t',1)

  #Parse key/value input
  try:
    row, col = map(int, key.split(','))
    value = value.split(',')
    key = (row, col)
    replicate_key, element_value = int(value[1]), float(value[2])
  except:
    continue

  #If we are still on the same key...
  if key == current_key:

    #Process key/value pair
    if replicate_key not in value_dict:
      value_dict[replicate_key] = [element_value]
    else:
      value_dict[replicate_key].append(element_value)

  #Otherwise, if this is a new key...
  else:

    #If this is a new key and not the first key we've seen
    if current_key:

      #compute/output result to STDOUT
      for j in range(n):
        if (j in value_dict) and (len(value_dict[j]) == 2):
          current_res += value_dict[j][0] * value_dict[j][1]
      print ('({0:d},{1:d}),{2:f}'.format(row, col, current_res))

    current_key = key
    value_dict = dict()

    #Process input for new key
    value_dict[replicate_key] = [element_value]
    current_res = 0.0


#Compute/output result for the last key

if current_key:
  for j in range(n):
    if (j in value_dict) and (len(value_dict[j]) == 2):
      current_res += value_dict[j][0] * value_dict[j][1]
  print('({0:d},{1:d}),{2:f}'.format(row, col, current_res))

In [ ]:
(matrix.txt)

A,0,0,0.0
A,0,1,1.0
A,0,2,2.0
A,0,3,3.0
A,0,4,4.0
A,1,0,5.0
A,1,1,6.0
A,1,2,7.0
A,1,3,8.0
A,1,4,9.0
B,0,0,0.0
B,0,1,1.0
B,0,2,2.0
B,1,0,3.0
B,1,1,4.0
B,1,2,5.0
B,2,0,6.0
B,2,1,7.0
B,2,2,8.0
B,3,0,9.0
B,3,1,10.0
B,3,2,11.0
B,4,0,12.0
B,4,1,13.0
B,4,2,14.0

# ***Create a python file mapper.py***
### Make the file executable

In [18]:
!dos2unix mapper.py

dos2unix: converting file mapper.py to Unix format...


In [19]:
!chmod +x mapper.py

# ***Create a python file reducer***
### Make the file executable

In [20]:
!dos2unix reducer.py

dos2unix: converting file reducer.py to Unix format...


In [21]:
!chmod +x reducer.py

# ***Run Matrix mulitplication through hadoop mapreduce***

In [22]:
!hadoop jar /content/hadoop-3.3.6/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar \
    -files mapper.py,reducer.py \
    -mapper mapper.py \
    -reducer reducer.py \
    -input input \
    -output output

2024-03-16 14:21:04,322 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2024-03-16 14:21:04,426 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2024-03-16 14:21:04,426 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2024-03-16 14:21:04,447 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2024-03-16 14:21:04,802 INFO mapred.FileInputFormat: Total input files to process : 1
2024-03-16 14:21:04,828 INFO mapreduce.JobSubmitter: number of splits:1
2024-03-16 14:21:05,032 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1355777797_0001
2024-03-16 14:21:05,032 INFO mapreduce.JobSubmitter: Executing with tokens: []
2024-03-16 14:21:05,463 INFO mapred.LocalDistributedCacheManager: Localized file:/content/mapper.py as file:/tmp/hadoop-root/mapred/local/job_local1355777797_0001_ad765d67-de8a-4eef-a2b1-e2e93e8445f1/mapper.py
2024-03-16 14:21:05,479 INFO mapred.LocalDistributedCacheMa

In [23]:
!cat output/part-00000

(0,1),90.000000	
(0,2),100.000000	
(1,0),110.000000	
(1,1),240.000000	
(1,2),275.000000	
(1,2),310.000000	
